In [1]:
import os
import cx_Oracle
import pandas as pd
import numpy as np
import pickle
from datetime import date,datetime
from sklearn import metrics
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score 
import time
import matplotlib.pyplot as plt
from sklearn.utils import resample
import datetime

In [2]:
os.chdir(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\Common Maps KNN\Data Files")

### Query for Data Download

In [ ]:
# drop table bd_geo_delinq;
# create table bd_geo_delinq as
# select * from
# (
# select a.skp_credit_case, a.DATE_decision, a.latitude as cus_lat, a.longitude as cus_long, grid_id,
# b.RISK_FPD10, b.RISK_FPD30, b.RISK_FSTQPD30, b.client_inr_income,
# b.PRODUCT_GROUP,case when score_cb<300 then 'NTC' else 'Others' end  as NTC_FLAG,  
# row_number() over (partition by a.skp_credit_case order by a.DATE_decision asc) as rnk
# from AP_BI.bd_geo a 
# join  
# (select * from
# uw_daily_data_all 
# where time_decision_date >= trunc(sysdate)-500
# )b
# on a.skp_credit_case = b.skp_credit_case 
# where a.date_decision >= trunc(sysdate)-500
# and a.latitude is not null 
# and a.longitude is not null 
# and a.result = 'FOUND'
# and lower(DECISION_OVERALL) = 'approved'
# ) a
# where rnk = 1
# ;

In [3]:
df= pd.read_pickle(os.getcwd()+r"\prod_delinq_comb.pkl")
df["DATE_DECISION"] = pd.to_datetime(df.DATE_DECISION, format = '%d-%m-%y')

In [8]:
df = df[["SKP_CREDIT_CASE","DATE_DECISION","CUS_LAT","CUS_LONG","RISK_FSTQPD30",
"GRID_ID","CLIENT_INR_INCOME"]]

In [10]:
df["MONTH_STR"] = df["DATE_DECISION"].dt.strftime("%Y-%m")

In [11]:
df = df[df["CLIENT_INR_INCOME"]<df["CLIENT_INR_INCOME"].quantile(0.99)]

In [12]:
df_month = pd.DataFrame({'month':[3],'year':[2020]}) #input the last month for which data is downloaded

### Download all active Grids

In [ ]:
# select grid_id, avg(cus_lat), avg(cus_long) from bd_geo_delinquency
# group by grid_id
# ;

In [14]:
all_grids= pd.read_csv(os.getcwd()+r"\grid id lat long.csv")
all_grids.columns = ["GRID_ID","CUS_LAT","CUS_LONG"]

In [17]:
df_final_test = pd.DataFrame()
df_final_grid = pd.DataFrame()

seed = 26

import datetime
from sklearn.neighbors import KNeighborsRegressor

for t in df_month.index:
    
    x=df_month["month"][t]
    year=df_month["year"][t]
    date_test_start = datetime.datetime(year, x, 1)
    date_test_end = (date_test_start +  datetime.timedelta(days=31)).replace(day=1)
    date_train_start = (date_test_start-datetime.timedelta(days=1)-datetime.timedelta(days=180)) 
    date_train_end = (date_test_start-datetime.timedelta(days=1))
    df_grid = all_grids
    df_grid["MONTH"] = date_test_start
    
    start_time = datetime.datetime.now()
    df_train = df[(df["DATE_DECISION"]>=date_train_start) & 
              (df["DATE_DECISION"]<date_train_end)]
    df_test = df[(df["DATE_DECISION"]>=date_test_start) & 
                  (df["DATE_DECISION"]<date_test_end)]

    
    df_train= df_train.sample(n=800000, random_state=seed)
    
    X_train = df_train[["CUS_LAT","CUS_LONG","DATE_DECISION"]]
    X_test = df_test[["CUS_LAT","CUS_LONG","DATE_DECISION"]]

    y_train = df_train["CLIENT_INR_INCOME"]
    y_test =  df_test["CLIENT_INR_INCOME"]

    knn = KNeighborsRegressor(n_neighbors=201)
    bst = knn.fit(X_train[["CUS_LAT","CUS_LONG"]], np.ravel(y_train))
    
    all_grids1 = all_grids
    included_grids = pd.DataFrame(df_train["GRID_ID"].unique())
    included_grids.columns = ['GRID_ID']
    all_grids1 = all_grids1.merge(included_grids, left_on = 'GRID_ID',right_on = 'GRID_ID', how = 'left',indicator = True)
    missing_grids=all_grids1[all_grids1["_merge"] != 'both'][["GRID_ID","CUS_LAT","CUS_LONG"]]
    df_train = pd.concat([df_train,missing_grids])

    ypred_train = bst.predict(df_train[["CUS_LAT","CUS_LONG"]])
    
    ypred_train1 = bst.predict(X_train[["CUS_LAT","CUS_LONG"]])   # Only for RMSE calculation
    ypred_test = bst.predict(X_test[["CUS_LAT","CUS_LONG"]])

    df_test['PRED_INCOME'] = ypred_test

    df_train ["PRED_INCOME"] = ypred_train
    grid_score = df_train[["GRID_ID","PRED_INCOME"]].groupby("GRID_ID").mean()["PRED_INCOME"].reset_index()
    grid_score.columns = ['GRID_ID',"PRED_INCOME"]
    df_grid =  pd.merge(df_grid, grid_score, on = 'GRID_ID', how = 'left')
    
    if df_final_test.shape == (0,0):
        df_final_test = df_test
    else:
        df_final_test = pd.concat([df_final_test,df_test],ignore_index=True)

    print("Training for predicting (month):",date_test_start.month, 
          "    RMSE TRAIN:",np.sqrt(metrics.mean_squared_error(y_train,ypred_train1)), 
          "    RMSE TEST:",np.sqrt(metrics.mean_squared_error(y_test,ypred_test)),
         "    time taken: ",'{:,.2}'.format((datetime.datetime.now()-start_time).seconds/60))

    if df_final_grid.shape == (0,0):
        df_final_grid = df_grid
    else:
        df_final_grid = pd.concat([df_final_grid,df_grid],ignore_index=True)




C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Training for predicting (month): 1     RMSE TRAIN: 6818.842360119617     RMSE TEST: 6722.346206639532     time taken:  3.9


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Training for predicting (month): 2     RMSE TRAIN: 6768.606993847599     RMSE TEST: 6730.5374989753645     time taken:  3.8


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

Training for predicting (month): 1     RMSE TRAIN: 7032.248938323342     RMSE TEST: 6793.024570620706     time taken:  3.8


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Training for predicting (month): 2     RMSE TRAIN: 6969.91560828149     RMSE TEST: 6758.036800122358     time taken:  3.6


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Training for predicting (month): 3     RMSE TRAIN: 6878.994234978258     RMSE TEST: 6735.157812521554     time taken:  3.5


In [24]:
df_final_test["PRED_INCOME"]=round(df_final_test["PRED_INCOME"])
df_final_grid["PRED_INCOME"]=round(df_final_grid["PRED_INCOME"])

In [25]:
df_final_test.to_pickle(os.getcwd()+r"\prod_income.pkl")
df_final_grid.to_pickle(os.getcwd()+r"\prod_grid_income.pkl")

In [27]:
df_final_grid[["GRID_ID","MONTH","PRED_INCOME"]].to_csv(os.getcwd()+r"\income_upload.csv", index = False)